In [ ]:
import pandas as pd
import numpy as np
import statsmodels.tools.tools as stattools
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from google.colab import files
uploaded = files.upload()

clothing_train = pd.read_csv("clothing_data_driven_training", sep="\t")
clothing_test = pd.read_csv("clothing_data_driven_test", sep="\t")

Saving clothing_data_driven_test to clothing_data_driven_test (1)
Saving clothing_data_driven_training to clothing_data_driven_training (1)


Using the training data set, create a C5.0 model (Model 1) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit. Obtain the predicted responses.

In [ ]:
# save target variable
y = clothing_train[['Response']]

# create dataset of predictors
X = clothing_train[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']]

# train C5.0 model
c50_01 = DecisionTreeClassifier(criterion="entropy", min_samples_split=75, max_leaf_nodes=5).fit(X,y)

# use model to predict on test data
c50_01_predict = c50_01.predict(clothing_test[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']])


Evaluate Model 1 using the test data set. Construct a contingency table to compare the actual and predicted values of Response.

In [ ]:
# construct contingency table
con_table = pd.crosstab(clothing_test['Response'], c50_01_predict)
con_table

col_0,0,1
Response,,
0,9173,441
1,1396,544


In [ ]:
def model_eval(matrix,model_name):
#from a dataframe and iloc is Purely integer-location based indexing for selection by position.
    tn = matrix.iloc[0,0]
    tp = matrix.iloc[1,1]
    fn = matrix.iloc[1,0]
    fp = matrix.iloc[0,1]
#some formulas
    tap = fn+tp
    tan = tn+fp
    tpn = tn+fn
    tpp = fp+tp
    precision = tp/tpp
    recall = tp/tap
    total = tn+tp+fn+fp
    data = [
	            round((tp+tn)/total,5),
	            round(1-((tp+tn)/total),5),
	            round(tp/tap,4),
	            round(tn/tan,4),
	            round(precision,4),
	            round(2 * (precision * recall) / (precision + recall),4),
	            round(5 * (precision * recall) / ((4 * precision) + recall),4),
	            round(1.25 * (precision * recall) / ((.25 * precision) + recall),4),
	            'Na',
	            'Na'
	            ]
    return(
	            pd.DataFrame(data, columns=[model_name],
	                         index=['Accuracy','Error Rate','Sensitivity','Specificity','Precision','F1','F2','F3','overall_cost','profit_per_customer'])
	            )

Model_1 = model_eval(con_table, model_name = 'Model_1')
Model_1


,Model_1
Accuracy,0.84101
Error Rate,0.15899
Sensitivity,0.2804
Specificity,0.9541
Precision,0.5523
F1,0.372
F2,0.311
F3,0.4626
overall_cost,Na
profit_per_customer,Na


Create a cost matrix, called the 4x cost matrix, that specifies a false positive is four times as bad as a false negative.

In [ ]:
cost_matrix_4x = {0:1, 1:4}

Using the training data set, build a C5.0 model (Model 2) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit, using the 4x cost matrix.

In [ ]:
# train C5.0 model
c50_02 = DecisionTreeClassifier(criterion="entropy", min_samples_split=75, max_leaf_nodes=5, class_weight=cost_matrix_4x).fit(X,y)

Evaluate your predictions from Model 2 using the actual response values from the test data set. Add Overall Model Cost and Profit per Customer to the Model Evaluation Table. Calculate all the measures from the Model Evaluation Table.

In [ ]:
# use model to predict on test data
c50_02_predict = c50_02.predict(clothing_test[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']])

# construct contingency table
con_table_2 = pd.crosstab(clothing_test['Response'], c50_02_predict)
con_table_2

col_0,0,1
Response,,
0,7004,2610
1,591,1349


In [ ]:


def model_eval_cost(matrix,model_name,fn_cost,tn_cost,fp_cost,tp_cost):
    tn = matrix.iloc[0,0]
    tp = matrix.iloc[1,1]
    fn = matrix.iloc[1,0]
    fp = matrix.iloc[0,1]
    tap = fn+tp
    tan = tn+fp
    tpn = tn+fn
    tpp = fp+tp
    precision = tp/tpp  if tpp != 0 else 0
    recall = tp/tap  if tap != 0 else 0
    total = tn+tp+fn+fp
    data = [
            round((tp+tn)/total,4),
            round(1-((tp+tn)/total),4),
            round(tp/tap,4),
            round(tn/tan,4),
            round(precision,4),
            round(2 * (precision * recall) / (precision + recall),4) if (precision + recall) !=0 else 'Na',
            round(5 * (precision * recall) / ((4 * precision) + recall),4) if (precision + recall) !=0 else 'Na',
            round(1.25 * (precision * recall) / ((.25 * precision) + recall),4) if (precision + recall) !=0 else 'Na',
            round((fn*fn_cost)+(tn*tn_cost)+(fp*fp_cost)+(tp*tp_cost),2),
            round(((fn*fn_cost)+(tn*tn_cost)+(fp*fp_cost)+(tp*tp_cost))/total,2)
            ]

    return(
            pd.DataFrame(data, columns=[model_name],
                         index=['Accuracy','Error Rate','Sensitivity','Specificity','Precision','F1','F2','F3','overall_cost','profit_per_customer'])
            )

Model_2 = model_eval_cost(con_table_2, model_name = 'Model_2', fn_cost=0, tn_cost=0, fp_cost=10, tp_cost=-40)
# Model_2 = model_eval_cost(con_table_2, model_name = 'Model_2', fn_cost=4, tn_cost=0, fp_cost=1, tp_cost=0)
Model_2


,Model_2
Accuracy,0.7230
Error Rate,0.2770
Sensitivity,0.6954
Specificity,0.7285
Precision,0.3407
F1,0.4574
F2,0.5756
F3,0.3794
overall_cost,-27860.0000
profit_per_customer,-2.4100


Upon completing the code and data sets that were posted under this week, continue using the below codes to construct the simplified data‐driven cost matrix as follows:
a. Obtain the mean of the Sales per Visit variable from the training data set and set the negative of that value to be the “cost” of a true positive. Let the false positive cost equal $10.

In [ ]:
# a
sales_mean = round(clothing_train['Sales per Visit'].mean(),2)
tp_cost = sales_mean - (sales_mean*2)
fp_cost = 10
sales_mean

113.58

b.	Construct the appropriate cost matrix, and simplify it, to obtain the simplified data‐driven cost matrix.

In [ ]:
data = {'0': [0, 0],
        '1': [fp_cost, tp_cost]
        }
cost_matrix = pd.DataFrame (data, columns = ['0','1'])

cost_matrix_adj = cost_matrix / fp_cost

cost_matrix_adj.iloc[1,0] = cost_matrix_adj.iloc[1,1] - (cost_matrix_adj.iloc[1,1]*2)
cost_matrix_adj.iloc[1,1] = 0

Using the training set, build a C5.0 model (Model 3) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit, using the data‐driven cost matrix.

In [ ]:
cost_matrix = {0:cost_matrix_adj.iloc[0,1], 1:cost_matrix_adj.iloc[1,0]}

# train C5.0 model
c50_03 = DecisionTreeClassifier(criterion="entropy", min_samples_split=75 , max_leaf_nodes=5 , class_weight=cost_matrix).fit(X,y)

**Populate** the Model Evaluation Table with the evaluation measures for Model 3, using the data‐driven cost matrix.

In [ ]:
# use model to predict on test data
c50_03_predict = c50_03.predict(clothing_test[['Days since Purchase', 'Number of Purchase Visits', 'Sales per Visit']])

# construct contingency table
con_table_3 = pd.crosstab(clothing_test['Response'], c50_03_predict)
con_table_3

col_0,0,1
Response,,
0,4897,4717
1,266,1674


In [ ]:
Model_3 = model_eval_cost(con_table_3, model_name = 'Model_3', fn_cost=0 , tn_cost=0 , fp_cost=fp_cost , tp_cost=tp_cost)
Model_3

,Model_3
Accuracy,0.5687
Error Rate,0.4313
Sensitivity,0.8629
Specificity,0.5094
Precision,0.2619
F1,0.4019
F2,0.5915
F3,0.3043
overall_cost,-142962.9200
profit_per_customer,-12.3700


In [ ]:
import datetime
import socket
def get_Host_name_IP():
    try:
        host_name = socket.gethostname()
        host_ip = socket.gethostbyname(host_name)
        print("Hostname:",host_name)
        print("IP Address:",host_ip)
    except:
        print("No visible IP Address")
get_Host_name_IP()
now = datetime.datetime.now()
print ("Time Stamp:", now.strftime("%Y-%m-%d %H:%M:%S"))

Hostname: 6b57865e5ae3
IP Address: 172.28.0.2
Time Stamp: 2023-01-10 23:43:14
